In [1]:
import pandas as pd
import numpy as np
import re  # Added for regex
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding


In [2]:
data = pd.read_csv('/content/swiggy.csv')
print("Columns in the dataset:")
print(data.columns)

Columns in the dataset:
Index(['ID', 'Area', 'City', 'Restaurant Price', 'Avg Rating', 'Total Rating',
       'Food Item', 'Food Type', 'Delivery Time', 'Review'],
      dtype='object')


#Text Cleaning and Sentiment Labeling

In [3]:
data["Review"] = data["Review"].str.lower()
data["Review"] = data["Review"].replace(r'[^a-z0-9\s]', '', regex=True)

data['sentiment'] = data['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0)
data = data.dropna()

#Tokenization and Padding

In [4]:
max_features = 5000
max_length = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data["Review"])
X = pad_sequences(tokenizer.texts_to_sequences(data["Review"]), maxlen=max_length)
y = data['sentiment'].values

#Data Splitting

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

#Build RNN Model

In [6]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=16),
    SimpleRNN(64, activation='tanh', return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#Train & Evaluate Model

In [7]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")


Epoch 1/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.6759 - loss: 0.6269 - val_accuracy: 0.7156 - val_loss: 0.5964
Epoch 2/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.7196 - loss: 0.5945 - val_accuracy: 0.7156 - val_loss: 0.5963
Epoch 3/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.7127 - loss: 0.5991 - val_accuracy: 0.7156 - val_loss: 0.6077
Epoch 4/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.7083 - loss: 0.6088 - val_accuracy: 0.7156 - val_loss: 0.5970
Epoch 5/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.7127 - loss: 0.5997 - val_accuracy: 0.7156 - val_loss: 0.5964
Epoch 6/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.7020 - loss: 0.6090 - val_accuracy: 0.7156 - val_loss: 0.5960
Epoch 7/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.7221 - loss: 0.5904 - val_accuracy: 0.7156 - val_loss: 0.5990
Epoch 8/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.7141 - loss: 0.5992 - val_

#Predicting Sentiment

In [8]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"

sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: The food was great.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
Sentiment: Positive (Probability: 0.71)
